# Check cell properties stats

# Setup

Setup Jupyter notebook's kernel based on python env setup with `requirements_silico.txt` (see comments in file for setup)

In [1]:
# SETUP PACKAGES
%load_ext autoreload
%autoreload 2
import os
import pprint as pp
import pandas as pd
import yaml
import numpy as np
from matplotlib import pyplot as plt
import spikeinterface as si
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as spost
from probeinterface.plotting import plot_probe
from spikeinterface.comparison import GroundTruthStudy
import shutil

pd.set_option('display.max_colwidth', None)

# SET PATHS
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting"

os.chdir(PROJ_PATH)
from src.nodes.io.silico.loading import load_campaign_params
from src.nodes.truth.silico import ground_truth
from src.nodes.io.silico import loading_raw_recording
from src.nodes.prepro import preprocess
from src.nodes.utils import get_config
from src.nodes.postpro import waveform
from src.nodes.dataeng.silico.filtering import filter_microcircuit_cells, get_cell_id_spiking_above_thresh, get_most_active_cell

# SET PARAMETERS
EXPERIMENT = "supp/silico_reyes"    # specifies the experiment 
SIMULATION_DATE = "2023_01_13" # specifies the run (date)

# set neuron filtering
MIN_SPIKE = 0
MS_BEFORE = 3                                
MS_AFTER = 3
CHANNEL_IDS = np.arange(64,80).tolist()   # plot traces from channels nearest to filtered neuron group


# SETUP CONFIG
data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()

2023-02-10 13:28:31,324 - root - utils.py - get_config - INFO - Reading experiment config.
2023-02-10 13:28:31,361 - root - utils.py - get_config - INFO - Reading experiment config. - done


**Basic cell stats**

In [2]:
# load campaign parameters
simulation = load_campaign_params(data_conf)

# get microcircuit's 30K cells
microcircuit_cells = filter_microcircuit_cells(simulation)
print("Microcircuit cells (count): ", len(microcircuit_cells["gid"]))

# get Spikeinterface's Sorting object for near-probe cells
output = ground_truth.run(simulation, data_conf, param_conf)
SortingObject = output["ground_truth_sorting_object"]
print("- Near-probe cell (count): ", SortingObject.get_num_units())

# get cell properties
cell_location = simulation["circuit"].cells.get(
    SortingObject.unit_ids, properties=["x", "y", "z"]
)

# get active cells (spike count above MIN_SPIKE)
active_cell_gids = get_cell_id_spiking_above_thresh(SortingObject, min_spike=MIN_SPIKE)
print("- near-probe active cell (count): ", len(active_cell_gids))

# get cell morph_class
cell_morph_class = simulation["circuit"].cells.get(
    SortingObject.unit_ids, properties=["morph_class"]
)
print("- near-probe morph classes (count): ", np.unique(cell_morph_class))

# get pyramidal cells
pyramidal_gids = cell_morph_class[cell_morph_class["morph_class"] == "PYR"].index.values
print("- near-probe pyramidal cell (count): ", len(pyramidal_gids))

# get active pyramidal cells
active_pyramidal_gids = list(set(pyramidal_gids) & set(active_cell_gids))
print("- near-probe active pyramidal cell (count): ", len(active_pyramidal_gids))

# get interneuron cells
interneuron_gids = cell_morph_class[
    cell_morph_class["morph_class"] == "INT"
].index.values
print("- near-probe interneuron cell (count): ", len(interneuron_gids))

# get active interneuron cells
active_interneuron_gids = list(set(interneuron_gids) & set(active_cell_gids))
print("- near-probe active interneuron cell (count): ", len(active_interneuron_gids))

# get the most active pyramidal cell and its spike count
most_active_pyr_gid = get_most_active_cell(SortingObject, cell_id=pyramidal_gids)
most_active_pyr_spike_count = SortingObject.get_total_num_spikes()[most_active_pyr_gid]
print("- near-probe most active pyramidal cell id: ", most_active_pyr_gid)
print(
    "- near-probe most active pyramidal cell spike (count): ",
    most_active_pyr_spike_count,
)

/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:51: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  Bool8 = np.bool8
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:54: DeprecationWarning: `np.object0` is a deprecated alias for ``np.object0` is a deprecated alias for `np.object_`. `object` can be used instead.  (Deprecated NumPy 1.24)`.  (Deprecated NumPy 1.24)
  Object0 = np.object0
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:66: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  Int0 = np.int0
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:80: DeprecationWarning: `np.uint0` is a deprecated alias for `np.uin

Microcircuit cells (count):  30190
- Near-probe cell (count):  1221
- near-probe active cell (count):  1221
- near-probe morph classes (count):  ['INT' 'PYR']
- near-probe pyramidal cell (count):  1090
- near-probe active pyramidal cell (count):  1090
- near-probe interneuron cell (count):  131
- near-probe active interneuron cell (count):  131
- near-probe most active pyramidal cell id:  3754013
- near-probe most active pyramidal cell spike (count):  106


# References

(1) https://probeinterface.readthedocs.io/en/stable/examples/ex_05_device_channel_indices.html 

(2) https://spikeinterface.readthedocs.io/en/latest/modules/core/plot_4_waveform_extractor.html 